# Dogs vs cats

In [1]:
import utils

import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import RMSprop

Using Theano backend.
Using gpu device 0: GeForce GTX 850M (CNMeM is disabled, cuDNN 5105)
/home/ursa/lib/anaconda2/envs/cv/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
PATH = 'data/sample'

In [3]:
utils.move_validset_into_trainset(PATH, verbose=True)

250	cat
250	dog


In [4]:
split_valid = 0.1
utils.generate_validation_set(PATH, val_split=split_valid, verbose=True)

250/2500		cat
250/2500		dog


In [5]:
img_shape = (3,224,224)

# Data Augmentation

In [6]:
rotation = 180
width_shift = 0.2
height_shift = 0.2
vertical_flip = True
horizontal_flip = True


datagen = ImageDataGenerator(
    rotation_range=rotation,
    width_shift_range=width_shift,
    height_shift_range=height_shift,
    vertical_flip=vertical_flip,
    horizontal_flip=horizontal_flip
)

# Model

In [7]:
batch_size = 4
nb_epoch = 5

optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'

In [8]:
model = Sequential()


In [9]:
model.add(BatchNormalization(axis=1, input_shape=img_shape))

model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(256, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='sigmoid'))

In [10]:
model.compile(
    optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [11]:
batches_train = datagen.flow_from_directory(
    PATH+'/train',
    target_size=(img_shape[1], img_shape[2]),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size
)

batches_val = datagen.flow_from_directory(
    PATH+'/valid',
    target_size=(img_shape[1], img_shape[2]),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size
)

Found 4500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [12]:
model.fit_generator(
    batches_train,
    batches_train.nb_sample,
    nb_epoch,
    validation_data=batches_val,
    nb_val_samples=batches_val.nb_sample
)

Epoch 1/5
4500/4500 [==============================] - 393s - loss: 0.6806 - acc: 0.5520 - val_loss: 0.6860 - val_acc: 0.5680
Epoch 2/5
4500/4500 [==============================] - 451s - loss: 0.6629 - acc: 0.6064 - val_loss: 0.6857 - val_acc: 0.5820
Epoch 3/5
4500/4500 [==============================] - 428s - loss: 0.6617 - acc: 0.6151 - val_loss: 0.7308 - val_acc: 0.6090
Epoch 4/5
4500/4500 [==============================] - 420s - loss: 0.6469 - acc: 0.6364 - val_loss: 0.7084 - val_acc: 0.6160
Epoch 5/5
4500/4500 [==============================] - 415s - loss: 0.6307 - acc: 0.6517 - val_loss: 0.6514 - val_acc: 0.6450
